In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.linalg import svd
from scipy.ndimage import gaussian_filter1d
import scipy as sp 
from scipy.signal import welch
import plotly.graph_objects as go
from scipy.signal import butter, lfilter

# Noise

Generate noise

In [ ]:
def noise_psd(N, psd = lambda f: 1):
    X_white = np.fft.rfft(np.random.randn(N))
    S = psd(np.fft.rfftfreq(N))
    S =  S/np.linalg.norm(S)
    X_shaped = X_white * S
    return np.fft.irfft(X_shaped)

def PSDGenerator(f):
    return lambda N: noise_psd(N, f)

@PSDGenerator
def white_noise(f):
    return 1

@PSDGenerator
def blue_noise(f):
    return np.sqrt(f)

@PSDGenerator
def violet_noise(f):
    return f

@PSDGenerator
def brownian_noise(f):
    return 1/np.where(f == 0, float('inf'), f)

@PSDGenerator
def pink_noise(f):
    return 1/np.where(f == 0, float('inf'), np.sqrt(f))

def gaussian_noise(f):
    return np.random.normal(size=f)

In [ ]:
plt.figure(figsize=(10, 6))

for label, kernel in zip(["white noise","blue_noise","violet noise","brownian noise","pink noise","gaussian noise"] , [white_noise,blue_noise,violet_noise,brownian_noise,pink_noise,gaussian_noise]):
    noise_signal = kernel(10000)
    frequencies, psd = welch(noise_signal, fs=20000, nperseg=1024)
    plt.semilogy(frequencies, psd,label= label)
plt.title('PSD')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power/Frequency (dB/Hz)')
plt.grid()
plt.legend()
plt.show()
    

# SSA

In [ ]:
def matrix_to_time_series(matrix, step_size,real_length):
    num_windows, window_size = matrix.shape
    time_series_length = window_size + (num_windows - 1) * step_size
    time_series = np.zeros(time_series_length)
    overlap_count = np.zeros(time_series_length)

    for i in range(num_windows):
        start_index = i * step_size
        end_index = start_index + window_size
        time_series[start_index:end_index] += matrix[i, :]
        overlap_count[start_index:end_index] += 1
    overlap_count[overlap_count == 0] = 1
    
    time_series /= overlap_count
    return time_series[:real_length]
def time_series_to_matrix(time_series, window_size, step_size):
    num_windows = max(1, int(np.ceil((len(time_series) - window_size) / step_size)) + 1)
    matrix = np.zeros((num_windows, window_size))
    
    for i in range(num_windows):
        start_index = i * step_size
        end_index = start_index + window_size
        if end_index <= len(time_series):
            matrix[i, :] = time_series[start_index:end_index]
        else:
            matrix[i, :len(time_series) - start_index] = time_series[start_index:len(time_series)]
            
    return matrix
def gernerate_k(prercent_k,window_size,step_size):
    return int(min(time_series_to_matrix(np.random.random(1000),200,2).shape) * 1)
def denoise_signal_svd(x,window_size,step_size,k):
    A = time_series_to_matrix(x, window_size, step_size)
    U, S, Vt = svd(A, full_matrices=False)
    S_p = np.zeros((k, k))
    np.fill_diagonal(S_p, S[:k])
    A_p = U[:, :k] @ S_p @ Vt[:k, :]
    x_p = matrix_to_time_series(A_p,step_size,len(x))
    per = sum(S[:k]) / sum(S)
    return x_p,per


def SNR(S, S_p):
    signal_norm = np.linalg.norm(S_p)
    noise_norm = np.linalg.norm(S - S_p)
    snr = 20 * np.log10(signal_norm / noise_norm) if noise_norm != 0 else np.inf
    return snr
def gernerate_k(prercent_k,window_size,step_size):
    return int(min(time_series_to_matrix(np.random.random(1000),window_size,step_size).shape) * prercent_k)



In [ ]:
a = [x for x in range(20)]
print(a)

In [ ]:
matrix =  time_series_to_matrix(a,10,1)
print("Transform a timeseries into matrix using sliding window:")
print(matrix)

In [ ]:
ts =  matrix_to_time_series(matrix,1,20)
print("Convert a matrix to timeseries:")
print(ts)

# Apply to denoise a simple signal

In [ ]:
time = np.linspace(0,10,1000)
simple_samples = np.cos(2*np.pi*time)

In [ ]:
plt.plot(simple_samples, label='Original')
plt.xlabel('Time')
plt.ylabel('amplitude')
plt.show()

In [ ]:
noise_signal_samples = simple_samples+ gaussian_noise(len(simple_samples))*0.1
plt.plot(noise_signal_samples, label='Signal with noise')
plt.xlabel('Time')
plt.ylabel('amplitude')
plt.show()

## The influence of some elements on svd

In [ ]:

wd_s_list = [x for x in range(10,500+1)]
prercent_k = 0.2
corrcoef_list = []
info_list = []
for wds in wd_s_list:
    k = gernerate_k(prercent_k,wds,1)
    dn_signal,info =  denoise_signal_svd(noise_signal_samples,wds,1,10)
    info_list.append(info)
    corrcoef_list.append(np.corrcoef(simple_samples,dn_signal )[0,1]) #correlation coefficients

In [ ]:
plt.plot(wd_s_list,corrcoef_list)
plt.title('Effect of window size on SVD')
plt.xlabel('window size')
plt.ylabel('correlation coefficients')
plt.show()

In [ ]:
plt.plot(wd_s_list,info_list)
plt.title('Effect of window size on SVD')
plt.xlabel('window size')
plt.ylabel('proportion of Information')
plt.show()

In [ ]:
step_list = [x for x in range(1,50+1)]
prercent_k = 0.5
corrcoef_list = []
info_list = []
for st in step_list:
    # k = gernerate_k(prercent_k,100,st)
    dn_signal,info =  denoise_signal_svd(noise_signal_samples,100,st,10)
    info_list.append(info)
    corrcoef_list.append(np.corrcoef(simple_samples,dn_signal )[0,1]) #correlation coefficients

In [ ]:
plt.plot(step_list,corrcoef_list)
plt.title('Effect of step  on SVD')
plt.xlabel('step')
plt.ylabel('correlation coefficients')
plt.show()

In [ ]:
plt.plot(step_list,info_list)
plt.title('Effect of step  on SVD')
plt.xlabel('step')
plt.ylabel('proportion of Information')
plt.show()

In [ ]:
k_list = [x for x in range(1,gernerate_k(1,50,5) +1)]
corrcoef_list = []
info_list = []
for k in k_list:
    dn_signal,info =  denoise_signal_svd(noise_signal_samples,100,1,k)
    info_list.append(info)
    corrcoef_list.append(np.corrcoef(simple_samples,dn_signal )[0,1]) #correlation coefficients

In [ ]:
plt.plot(k_list,corrcoef_list)
plt.title('Effect of k  on SVD')
plt.xlabel('k')
plt.ylabel('correlation coefficients')
plt.show()

In [ ]:
plt.plot(k_list,info_list)
plt.title('Effect of step  on SVD')
plt.xlabel('k')
plt.ylabel('proportion of Information')
plt.show()

## Denoising with good parameters

In [ ]:
denoised_signal,per =denoise_signal_svd(noise_signal_samples,50,5,4)
plt.plot(denoised_signal, label='Smoothed')
plt.plot(simple_samples, label='Original')

plt.xlabel('Time')
plt.ylabel('amplitude')
plt.legend()
plt.show()

In [ ]:
print(f"Correlation coefficients: { np.corrcoef(noise_signal_samples,simple_samples)[1,0]}")
print(f"Proportion of information: { per}")


# Apply to denoise a digital audio 


## EDA + preprocessing

A conversation between two people.

In [ ]:
samples, sample_rate = librosa.load("data/sample2.wav", sr=None)


In [ ]:
Audio(samples,rate=sample_rate)

In [ ]:
plt.figure(figsize=(14, 10))
plt.subplot(2, 1, 1)
plt.plot(samples)
plt.title(label='Original Data')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
S = np.abs(librosa.stft(samples))
plt.subplot(2, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sample_rate, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

In [ ]:
noise_sample = samples + gaussian_noise(len(samples))*0.01

In [ ]:
print(f"Correlation coefficients: { np.corrcoef(samples,noise_sample)[1,0]}")
print( f'Signal to Noise Ratio :{SNR(samples,noise_sample)}')

In [ ]:
Audio(noise_sample,rate=sample_rate)

In [ ]:
plt.figure(figsize=(14, 10))
plt.subplot(2, 1, 1)
plt.plot(noise_sample)
plt.title(label='Data with noise')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
S = np.abs(librosa.stft(noise_sample))
plt.subplot(2, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sample_rate, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.plot(noise_sample, label='Noise Data')
plt.plot(samples, label='Original Data')
plt.title(label='Wave plot')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

## Svd on total signal

In [ ]:
smoothed_data,per =denoise_signal_svd(noise_sample,1024,256,70)


In [ ]:
print(f"Correlation coefficients: { np.corrcoef(samples,smoothed_data)[1,0]}")
print(f"Proportion of information: { per}")
print( f'Signal to Noise Ratio :{SNR(samples,smoothed_data)}')

In [ ]:
Audio(smoothed_data,rate=sample_rate)

In [ ]:
plt.figure(figsize=(14, 10))
plt.subplot(2, 1, 1)
plt.plot(smoothed_data)
plt.title(label='Denoised Data')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
S = np.abs(librosa.stft(smoothed_data))
plt.subplot(2, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sample_rate, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.plot(samples[:50000], label='Original Data')
plt.plot(smoothed_data[:50000], label='Smoothed')

plt.xlabel('Time')
plt.ylabel('amplitude')
plt.legend()
plt.show()

In [ ]:
plt.plot(noise_sample[20000:50000], label='noise_sample')
plt.plot(smoothed_data[20000:50000], label='Smoothed')
plt.title(label='Wave plot')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=smoothed_data[26000:28000], mode='lines', name='Denoised'))
fig.add_trace(go.Scatter( y=noise_sample[26000:28000], mode='lines', name='Noise'))
fig.add_trace(go.Scatter( y=samples[26000:28000], mode='lines', name='Original'))


fig.update_layout(
    title='Interactive wave Plot',
    xaxis_title='Time',
    yaxis_title='amplitude',
    xaxis_rangeslider_visible=True
)

fig.show()

## Partitoning denoising

In [ ]:
t = len(noise_sample)
i = 0
step = sample_rate*4
p_denoised_signal = np.array([])
info_list= np.array([])
while i < t:
    trim_noise_sample = noise_sample[i:i+step]
    segment,per =denoise_signal_svd(trim_noise_sample,1024,128,60)
    p_denoised_signal = np.append(p_denoised_signal, segment)
    info_list = np.append(info_list, per)
    i+=step  


In [ ]:
print(f"Correlation coefficients: { np.corrcoef(samples,p_denoised_signal)[1,0]}")
print(f"Average proportion of information: { np.mean(info_list)}")
print( f'Signal to Noise Ratio :{SNR(samples,p_denoised_signal)}')

In [ ]:
Audio(p_denoised_signal,rate=sample_rate)

In [ ]:
plt.figure(figsize=(14, 10))
plt.subplot(2, 1, 1)
plt.plot(p_denoised_signal)
plt.title(label='Partitioning denoised Data')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
S = np.abs(librosa.stft(p_denoised_signal))
plt.subplot(2, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sample_rate, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.plot(samples[20000:50000], label='Original')
plt.plot(p_denoised_signal[20000:50000], label='Smoothed')
plt.title(label='Wave plot')
plt.xlabel('Time')
plt.ylabel('amplitude')
plt.legend()
plt.show()

In [ ]:
plt.plot(noise_sample[20000:50000], label='noise_sample')
plt.plot(p_denoised_signal[20000:50000], label='Smoothed')
plt.title(label='Wave plot')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=p_denoised_signal[:sample_rate*20], mode='lines', name='Denoised'))
fig.add_trace(go.Scatter( y=noise_sample[:sample_rate*20], mode='lines', name='Noise'))
fig.add_trace(go.Scatter( y=samples[:sample_rate*20], mode='lines', name='Original'))


fig.update_layout(
    title='Interactive Wave Plot',
    xaxis_title='Time',
    yaxis_title='amplitude',
    xaxis_rangeslider_visible=True
)

fig.show()

## Comparing total signal svd vs partitioning svd

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=smoothed_data[:sample_rate*20], mode='lines', name='Default denoise'))
fig.add_trace(go.Scatter( y=p_denoised_signal[:sample_rate*20], mode='lines', name='Partitioning denoise'))
fig.add_trace(go.Scatter( y=samples[:sample_rate*20], mode='lines', name='Original'))

fig.update_layout(
    title='Interactive wave Plot',
    xaxis_title='Time',
    yaxis_title='amplitude',
    xaxis_rangeslider_visible=True
)

fig.show()

In [ ]:
fft_result = np.fft.rfft(smoothed_data)
fft_spectrum_abs1 = np.abs(fft_result)
freq1 = np.fft.rfftfreq(len(smoothed_data), 1 / sample_rate)

fft_result = np.fft.rfft(p_denoised_signal)
fft_spectrum_abs2 = np.abs(fft_result)
freq2 = np.fft.rfftfreq(len(smoothed_data), 1 / sample_rate)



plt.semilogy(freq1, fft_spectrum_abs1,label ="Default denoise")
plt.semilogy(freq2, fft_spectrum_abs2,label ="Partitioning denoise")
plt.title('Power Spectral Density (PSD)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('log(Amplitude)')
plt.legend()
plt.show()

There is still a small amount of noise scattered throughout the frequencies that SVD cannot completely resolve

## Filtering

In [ ]:
fft_spectrum = np.fft.rfft(p_denoised_signal)
freq = np.fft.rfftfreq(len(p_denoised_signal), d=1./sample_rate)
fft_spectrum_abs = np.abs(fft_spectrum)
plt.plot(freq, fft_spectrum_abs)
plt.xlim(0,3000)
plt.title("Frequency Spectrum plot")
plt.xlabel("frequency, Hz")
plt.ylabel("Amplitude, units")
plt.show()

- Male voices: Generally range from 85 Hz to 180 Hz.
- Female voices: Generally range from 165 Hz to 255 Hz. \
So, we can filter the final signal between 20hz and 1000hz (It depend on Frequency Spectrum)

In [ ]:
fft_spectrum = np.fft.rfft(p_denoised_signal)
freq = np.fft.rfftfreq(len(p_denoised_signal), d=1./sample_rate)
for i,f in enumerate(freq):
    if f < 50 or f > 1000 :
        fft_spectrum[i] = 0.0
fp__denoised_signal = np.fft.irfft(fft_spectrum)

In [ ]:
Audio(fp__denoised_signal,rate=sample_rate)

In [ ]:
fft_spectrum = np.fft.rfft(fp__denoised_signal)
freq = np.fft.rfftfreq(len(fp__denoised_signal), d=1./sample_rate)
fft_spectrum_abs = np.abs(fft_spectrum)
plt.plot(freq, fft_spectrum_abs)
plt.xlabel("frequency, Hz")
plt.ylabel("Amplitude, units")
plt.show()

In [ ]:
plt.plot(samples)
plt.plot(fp__denoised_signal)

In [ ]:
plt.figure(figsize=(14, 10))
plt.subplot(2, 1, 1)
plt.plot(fp__denoised_signal)
plt.title(label='Partitioning denoised Data with filter')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
S = np.abs(librosa.stft(fp__denoised_signal))
plt.subplot(2, 1, 2)
librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), sr=sample_rate, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

# Modifying Model

In [ ]:
# Add a method choose k :the threshold index τ can be computed as the center of mass for the singular values.
# Add alpha: adjust small singular values 
# Add alpha: adjust large singular values 

def denoise_signal_svd_modified(x, window_size, step_size,alpha=0.1,beta =1, fraction=0.1):
    A = time_series_to_matrix(x, window_size, step_size)
    U, S, Vt = svd(A, full_matrices=False)
    tau = sum((i + 1) * S[i] for i in range(len(S))) / sum(S)
    tau_adjusted = fraction * tau
    M0 = int(tau_adjusted)
    S_p = np.zeros_like(S)
    S_p[:M0] = S[:M0]*beta
    S_p[M0:] = alpha * S[M0:]
    A_p = U @ np.diag(S_p) @ Vt
    x_p = matrix_to_time_series(A_p, step_size, len(x))
    per = sum(S_p**2) / sum(S**2)
    return x_p, per

## Using our teammate's recording

In [ ]:
samples, sample_rate = librosa.load("data/nonoise.wav", sr=None)

Add gaussian noise to clean signal

In [ ]:
noise_sample =samples+ gaussian_noise(len(samples))*0.01

In [ ]:
Audio(noise_sample,rate=sample_rate)

In [ ]:
smoothed_data,per =denoise_signal_svd_modified(noise_sample,1024,256,alpha =0.00,beta= 1,fraction=0.85)

In [ ]:
Audio(smoothed_data,rate=sample_rate) # signal after denoising, but still have a little bit of noise

In [ ]:
print(f"Correlation coefficients: { np.corrcoef(samples,smoothed_data)[1,0]}")
print(f"Proportion of information: { per}")
print( f'Signal to Noise Ratio :{SNR(samples,smoothed_data)}')
print( f'RMSE :{np.linalg.norm(samples - smoothed_data)/len(samples)**(1/2)}')

The noise signal extract with svd

In [ ]:
removed_signal,per =denoise_signal_svd_modified(noise_sample,1024,256,alpha =1,beta=0,fraction=0.85)

In [ ]:
Audio(removed_signal,rate=sample_rate)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( y=smoothed_data[26000:28000], mode='lines', name='Denoised'))
fig.add_trace(go.Scatter( y=noise_sample[26000:28000], mode='lines', name='Noise'))
fig.add_trace(go.Scatter( y=samples[26000:28000], mode='lines', name='Original'))
fig.update_layout(
    title='Interactive wave Plot',
    xaxis_title='Time',
    yaxis_title='amplitude',
    xaxis_rangeslider_visible=True
)

fig.show()

## Performance improvements

Method 1: Continue denoising with svd

In [ ]:
smoothed_lp = smoothed_data
for i in range(100):
    smoothed_lp,_ =denoise_signal_svd(smoothed_lp,1024,256,100)

In [ ]:
Audio(smoothed_lp,rate=sample_rate) 

Method 2: Using a filter and smooth with convolution

In [ ]:
def bandpass_filter(data, lowcut, highcut, fs, order=5): #Fillter function 
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

In [ ]:
kernel_size = 10
kernel = np.hamming(kernel_size)
# Human voice frequency is between 300 and 3000 Hz
filtered_signal = bandpass_filter(smoothed_data,300,4000,sample_rate)
filtered_signal = sp.signal.convolve(filtered_signal, kernel/kernel.sum(), mode='same') # Smooth signal after filtering

In [ ]:
Audio(filtered_signal,rate=sample_rate)

Combine 2 method

In [ ]:
filtered_signal = bandpass_filter(smoothed_lp,300,4000,sample_rate)
filtered_signal = sp.signal.convolve(filtered_signal, kernel/kernel.sum(), mode='same') 
Audio(filtered_signal,rate=sample_rate)